In [3]:
# import necessary libraries
import numpy as np 
import pandas as pd 
import os, sys
from pydantic import BaseModel
from typing import List, Tuple, Literal, Optional

from llm_model import ChatLLM
from prompts.prompt_v1 import get_prompt_v1

In [4]:
data = pd.read_csv(r'C:\Users\admin\Documents\Review Prompting\data\yelp_preprocessed.csv')
data.head(3)

,stars,text,cool,useful,funny
0,5,My wife took me here on my birthday for breakf...,2,5,0
1,5,I have no idea why some people give bad review...,0,0,0
2,4,love the gyro plate. Rice is so good and I als...,0,1,0


In [5]:
subset_data = data.head(100)
x_train, y_train = subset_data.drop(columns=['stars']), subset_data['stars']

# Setup
model = ChatLLM(provider='groq').model
prompt, parser = get_prompt_v1()

# Define Chain
chain = prompt | model | parser

In [6]:
import json

result = []
reviews = []
for i in range(3):
    reactions = {
        'cool': x_train.iloc[i]['cool'],
        'useful': x_train.iloc[i]['useful'],
        'funny': x_train.iloc[i]['funny']
    }
    res = chain.invoke({'review_text': x_train.iloc[i]['text'],
                    'reactions': reactions})
    reviews.append(res.predicted_stars)
    result.append(res)

print(result)
print(reviews)
print(y_train.values)

[StructuredOutput(predicted_stars=5, explanation="The review is extremely positive, with the user using words like 'excellent', 'phenomenal', 'amazing', and 'delicious' to describe their experience. They also mention that they can't wait to go back, which suggests a high level of satisfaction."), StructuredOutput(predicted_stars=5, explanation='The user had a very positive experience, mentioning pleasant staff, good prices, and awesome food, with no criticisms or negative comments.'), StructuredOutput(predicted_stars=5, explanation='The user uses positive language and expresses enthusiasm for multiple aspects of the product, including the gyro plate, rice, and candy selection, indicating a high level of satisfaction.')]
[5, 5, 5]
[5 5 4 5 5 4 5 4 4 5 5 5 5 4 4 2 3 5 3 4 3 5 5 1 5 4 5 4 4 4 5 1 5 4 3 1 4
 4 4 4 4 4 4 4 4 3 5 3 4 4 4 5 4 4 5 4 2 4 4 5 4 1 3 4 1 1 5 5 4 5 4 1 4 4
 5 4 4 4 5 1 4 4 5 4 1 4 5 4 5 1 1 5 4 4 4 5 4 4 5 5]
